In [ ]:
!pip install aspect_based_sentiment_analysis #installing module for aspect based sentiment analyisis

In [ ]:
# useful imports
import aspect_based_sentiment_analysis as absa
import pandas as pd
import os.path
from google.colab import drive
from pathlib import Path

In [ ]:
drive.mount('/content/drive', force_remount=True)

In [ ]:
# loading the nlp model for aspect based sentiment analysis
nlp = absa.load() 

In [ ]:
def sentiment_analysis(text_series, sentiments = {'Positive': 2, 'Neutral': 0, 'Negative': 1}, aspects = ['Appearance', 'Aroma', 'Palate', 'Taste']):
    """
    Function that does sentiment analyisis based on the text.

    Input:
        - text_series: pandas series containing textual reviews that users wrote
        - sentiments: dictionary of sentiments
        - aspects: list of aspects we're interested in

    Returns:
        - dataframe containing scores for each sentiment towards each aspect of 
      textual reviews from text_series.
    """
  
    # we're applying model to each textual review
    try:
        tasks = text_series.apply(lambda text: nlp(text, aspects=aspects)) 
    except :
        print('An error occured during sentiment analysis!')
        return
        
    # dictionary that maps each (aspect,sentiment) pair to a list containing scores of 
    # the specifc sentiment to the specific aspects for each of the given reviews
    columns = {} 
    for aspect in aspects:
        for sentiment_str, _ in sentiments.items():
          # initially, list is empty
          columns[aspect + ' ' + sentiment_str] = [] 

    for task in tasks:
        for aspect_index in range(len(aspects)):
            scores = task.examples[aspect_index].scores
            for sentiment_str, i in sentiments.items():
                columns[aspects[aspect_index] + ' ' + sentiment_str].append(scores[i])
  
    # converting dictionary to pandas dataframe
    return pd.DataFrame(columns).set_index(text_series.index)

In [ ]:
def sentiment_analysis_of_batch(batch_folder, output_folder, filename):
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    # Example of path on Google Colab: /content/drive/My Drive/batches/test.csv
    data = pd.read_csv(os.path.join(batch_folder, '{}.csv'.format(filename)))
    
    # getting sentiment analysis from the function
    columns = sentiment_analysis(data['Text'])
    
    # concatenating scores in each row
    expanded_data = pd.concat([data, columns], axis=1)
    
    # write the results
    expanded_data.to_csv(os.path.join(output_folder, '{}_done.csv'.format(filename)), index=False)

In [ ]:
# Check if it is the same path for you!
batch_folder = '/content/drive/My Drive/batches/'
output_folder = '/content/drive/My Drive/results/'

In [ ]:
# Example
# Change filename with file you want to work on!
filename = 'test'
sentiment_analysis_of_batch(batch_folder, output_folder, filename)